In [33]:
import os

In [34]:
%pwd
os.chdir("..")

In [35]:
%pwd

'c:\\Users\\user\\Desktop\\End-to-End-ML-project-MLflow'

In [1]:
import dagshub

dagshub.init(
    repo_owner='Francisroyce',
    repo_name='End-to-End-ML-project-MLflow',
    mlflow=True
)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\user\anaconda3\envs\myenv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=7bbb70eb-ba4b-44c9-ad68-72d176fc3993&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=cfb60bc75e4be9b309e80c08def9971047dd818d7e2bc1863a6627bf0b584ca3




Accessing as Francisroyce

Initialized MLflow to track repo "Francisroyce/End-to-End-ML-project-MLflow"

Repository Francisroyce/End-to-End-ML-project-MLflow initialized!

In [2]:
import mlflow

print(mlflow.get_tracking_uri())


https://dagshub.com/Francisroyce/End-to-End-ML-project-MLflow.mlflow


In [3]:
import mlflow

# Start a run
with mlflow.start_run():
    # Log parameters (hyperparameters)
    mlflow.log_param("learning_rate", 0.01)
    mlflow.log_param("optimizer", "adam")
    
    # Log metrics
    mlflow.log_metric("accuracy", 0.92)
    mlflow.log_metric("loss", 0.15)


🏃 View run whimsical-snipe-630 at: https://dagshub.com/Francisroyce/End-to-End-ML-project-MLflow.mlflow/#/experiments/0/runs/9a266470003f4f1e9a63fa4d703ee511
🧪 View experiment at: https://dagshub.com/Francisroyce/End-to-End-ML-project-MLflow.mlflow/#/experiments/0


In [ ]:
for run in mlflow.search_runs():
    print(run[['run_id', 'status', 'metrics.accuracy', 'metrics.loss']])


                             run_id    status  metrics.accuracy  metrics.loss
0  9a266470003f4f1e9a63fa4d703ee511  FINISHED              0.92          0.15


In [123]:
# entity

from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_url: str
  

In [124]:
# configuration manager

from pathlib import Path
from my_project.constants import * 
from my_project.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
        schema_filepath: Path = SCHEMA_FILE_PATH,
    ):
        # Load configs
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Ensure artifacts root exists
        create_directories([Path(self.config.artifacts_root)])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        # Attribute-style access for config
        config = self.config.model_evaluation

        # Use model evaluation params from params.yaml
        # This now exists in your updated params.yaml
        params = self.params.model_evaluation

        # Use target_column from schema.yaml
        schema = self.schema.target_column

        # Ensure evaluation root directory exists
        create_directories([Path(config.root_dir)])

        # Build ModelEvaluationConfig dataclass
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_params=params,  # now contains meaningful evaluation params
            metric_file_name=Path(config.metric_file_name),
            target_column=schema,
            mlflow_url=config.mlflow_url
        )

        return model_evaluation_config


In [ ]:
# component/model_evaluation.py
import os
import json
import joblib
import pandas as pd
import numpy as np
import mlflow
import uuid
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from my_project import logger


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

        # --- Create a unique experiment name to avoid deleted experiment errors ---
        experiment_name = f"model_evaluation_{uuid.uuid4().hex[:6]}"
        mlflow.set_experiment(experiment_name)

        # Ensure directories exist
        os.makedirs(self.config.root_dir, exist_ok=True)
        os.makedirs(Path(self.config.metric_file_name).parent, exist_ok=True)

    def evaluate_model(self):
        """
        Evaluate trained model on test data, log metrics to MLflow (DagsHub-compatible),
        save metrics locally, and save the model locally (artifact folder).
        """
        try:
            # Load test data
            test_df = pd.read_csv(self.config.test_data_path)
            if self.config.target_column not in test_df.columns:
                raise KeyError(f"Target column '{self.config.target_column}' not found in test data")

            x_test = test_df.drop(columns=[self.config.target_column])
            y_test = test_df[self.config.target_column]

            # Load trained model
            if not Path(self.config.model_path).exists():
                raise FileNotFoundError(f"Model file not found at {self.config.model_path}")
            model = joblib.load(self.config.model_path)

            # Generate predictions
            y_pred = model.predict(x_test)

            # Calculate metrics
            metrics = {
                "MAE": mean_absolute_error(y_test, y_pred),
                "MSE": mean_squared_error(y_test, y_pred),
                "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
                "R2_Score": r2_score(y_test, y_pred)
            }

            # --- LOG METRICS AND PARAMETERS TO MLflow ONLY ---
            with mlflow.start_run(run_name="model_evaluation"):
                if self.config.all_params:
                    for key, value in self.config.all_params.items():
                        mlflow.log_param(key, value)
                for key, value in metrics.items():
                    mlflow.log_metric(key, value)

            # --- SAVE METRICS LOCALLY ---
            with open(self.config.metric_file_name, "w") as f:
                json.dump(metrics, f, indent=4)

            # --- SAVE MODEL LOCALLY AS ARTIFACT ---
            artifact_dir = Path(self.config.root_dir) / "model_artifacts"
            artifact_dir.mkdir(exist_ok=True)
            joblib.dump(model, artifact_dir / "model.joblib")

            logger.info(f"Model evaluation metrics logged and saved: {metrics}")
            logger.info(f"Model saved locally at: {artifact_dir / 'model.joblib'}")
            return metrics

        except Exception as e:
            logger.error(f"Error during model evaluation: {e}")
            raise


In [126]:
# pipeline
import dagshub

# --- Initialize DagsHub and MLflow ---
dagshub.init(
    repo_owner='Francisroyce',
    repo_name='End-to-End-ML-project-MLflow',
    mlflow=True
)
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    metrics = model_evaluation.evaluate_model()
    print("Model evaluation metrics:", metrics)
except Exception as e:
    print(f"Error during model evaluation: {e}")

2025-09-01 15:39:36,792 - httpx - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/Francisroyce/End-to-End-ML-project-MLflow "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Francisroyce/End-to-End-ML-project-MLflow"

2025-09-01 15:39:36,823 - dagshub - INFO - Initialized MLflow to track repo "Francisroyce/End-to-End-ML-project-MLflow"


Repository Francisroyce/End-to-End-ML-project-MLflow initialized!

2025-09-01 15:39:36,834 - dagshub - INFO - Repository Francisroyce/End-to-End-ML-project-MLflow initialized!
2025-09-01 15:39:36,857 - my_project - INFO - YAML file 'config\config.yaml' read successfully.
2025-09-01 15:39:36,865 - my_project - INFO - YAML file 'params.yaml' read successfully.
2025-09-01 15:39:36,877 - my_project - INFO - YAML file 'schema.yaml' read successfully.
2025-09-01 15:39:36,883 - my_project - INFO - Directory created: 'artifacts'
2025-09-01 15:39:36,889 - my_project - INFO - Directory created: 'artifacts\model_evaluation'
Error during model evaluation: Cannot set a deleted experiment 'model_evaluation' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.
